<a href="https://www.nvidia.com/dli"> <img src="images/DLI Header.png" alt="Header" style="width: 400px;"/> </a>

# 在Python中利用Numba客製CUDA Kernel

在此段落中，我們將進一步探討CUDA 程式設計模型如何組織平行工作，並將利用這個知識撰寫出在CUDA GPU上平行運作的自訂CUDA **kernel**函數。使用CUDA程式設計模型自訂CUDA kernel比起單純用`@vectorize`修飾ufunc在實作上所需要更多工作流程。然而，這種kernel卻能在無法利用ufunc實現平行運算之發揮所長，並且提供支持最高性能的靈活性。

本段包含三篇可供您深入研究的附錄：有助於GPU程式設計的各種除錯技巧、CUDA程式設計參考資料連結以及GPU上的Numba支援隨機數字產生。

## 目標

完成本段後您將能夠：

* 以Python撰寫自訂CUDA kernel並以執行執行配置(Execution configuration)。
* 使用Grid跨度迴圈在大型資料集上平行運作並發揮記憶體合併。
* 利用原子操作(Atomic operations)避免平行工作時出現競爭條件(Race conditions)

## 自訂Kernel的需要

Ufunc十分簡練，並且對於任何應在資料元素方面執行的純量操作而言，ufunc是最適合任務所需的工具。

如您所知，許多問題不是只靠對資料集的各個元素套用相同函數就能解決。例如需要存取資料結構中一種以上元素以計算其輸出的問題，像是模板演算法(Stencil algorithms)，又或者是無法透過一個輸入值對一個輸出值映射來表達的問題，例如歸約(Reduction)。這類問題許多仍屬可平行性質，但無法使用ufunc來表達。

撰寫自訂CUDA Kernel雖然比起撰寫GPU加速ufunc來說更為困難，但在可送至GPU平行運作的函數類型上，能夠為開發人員提供極大的靈活性。此外，如本段及下段所將提及，這種方式也更能精細控制如何透過向開發人員明確揭露CUDA Thread階層而達成平行。

雖然完全保持在Python，但我們利用Numba撰寫CUDA Kernel的方法很像是開發人員在CUDA C/C++中使用的撰寫方法。若您熟悉使用CUDA C/C++程式設計，應該很快就能掌握使用Numba以Python自訂Kernel的技巧，而若您是初次學習，則此處所學會在您需要或想要以C/C++製作CUDA時，或甚至研究網路上各種最常表達CUDA C/C++程式碼的CUDA資源時很有幫助。

## CUDA kernel簡介

在CUDA中程式設計時，開發人員為GPU撰寫稱為**Kernel**的函數，在GPU的許多核心上以平行**執行緒(Thread)**的型態執行，或者，用CUDA角度來說是**啟動**。啟動kernel時，程式設計師使用的特殊語法稱為**執行配置(Execution configuration)**，用來描述平行執行的配置。

以下投影片(執行下方單元後出現)說明如何創造可於GPU裝置上平行處理大型資料集的CUDA Kernel。跟隨投影片操作，您就可運用投影片中講述的方式來開始撰寫並執行您自訂的CUDA kernel。

In [1]:
from IPython.display import IFrame
IFrame('https://view.officeapps.live.com/op/view.aspx?src=https://developer.download.nvidia.com/training/courses/C-AC-02-V1-TW/AC_CUDA_Python_1-TW.pptx', 640, 300)

## 第一個CUDA Kernel

現在我們以一個具體且十分簡單的範例開始，也就是重寫一維NumPy陣列的加法函數。CUDA Kernel是以`numba.cuda.jit`修飾器編譯。請不要將`numba.cuda.jit`與您之前看到用於最佳化**CPU**函數的`numba.jit`修飾器混淆。

我們先透過簡單的範例來強調一些必要語法。值得一提的是，這個函數其時也能寫成ufunc，但我們在此還是將焦點放在學習語法上。我們將於下文繼續討論更適合寫成自訂Kernel的函數。註解中有關於程式碼的重要資訊，請務必仔細閱讀。

In [2]:
from numba import cuda

# Note the use of an `out` array. CUDA kernels written with `@cuda.jit` do not return values,
# just like their C counterparts. Also, no explicit type signature is required with @cuda.jit
@cuda.jit
def add_kernel(x, y, out):
    
    # The actual values of the following CUDA-provided variables for thread and block indices,
    # like function parameters, are not known until the kernel is launched.
    
    # This calculation gives a unique thread index within the entire grid (see the slides above for more)
    idx = cuda.grid(1)          # 1 = one dimensional thread grid, returns a single value.
                                # This Numba-provided convenience function is equivalent to
                                # `cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x`

    # This thread will do the work on the data element with the same index as its own
    # unique index within the grid.
    out[idx] = x[idx] + y[idx]

In [3]:
import numpy as np

n = 4096
x = np.arange(n).astype(np.int32) # [0...4095] on the host
y = np.ones_like(x)               # [1...1] on the host

d_x = cuda.to_device(x) # Copy of x on the device
d_y = cuda.to_device(y) # Copy of y on the device
d_out = cuda.device_array_like(d_x) # Like np.array_like, but for device arrays

# Because of how we wrote the kernel above, we need to have a 1 thread to one data element mapping,
# therefore we define the number of threads in the grid (128*32) to equal n (4096).
threads_per_block = 128
blocks_per_grid = 32

In [4]:
add_kernel[blocks_per_grid, threads_per_block](d_x, d_y, d_out)
cuda.synchronize()
print(d_out.copy_to_host()) # Should be [1...4096]

[   1    2    3 ... 4094 4095 4096]


### 練習：調整程式碼

稍微改變上面的程式碼，觀察對其執行有何影響。執行程式碼前，先猜猜看會發生什麼改變：

* 減少`threads_per_block`變數
* 減少`blocks_per_grid`變數
* 增加`threads_per_block`及/或`blocks_per_grid` 變數
* 去除或註解排除`cuda.synchronize()`呼叫

### 結果

上面的範例中，由於Kernel的寫法是讓每一個Thread確實處理一個資料元素，所以Grid中的Thread數量必定會等於資料元素數量。

減少Block數量，及/或減少每一Block的Thread數量，藉此**減少Grid中的Thread數量**，就會留有尚未處理到的元素，因此我們可以在輸出中看到朝向`d_out`陣列末端的元素沒有加上任何數值。若您是藉由減少每一Block中Thread數量的方式來編輯執行配置，則會有其他通過`d_out`陣列的元素未經處理。

**增加Grid尺寸**事實上會產生錯誤。本段稍後將說明如何發現錯誤並加以修正。

您可能以為**去除同步點**的話，印出結果會顯示沒有完成任何工作或只完成一點工作。這是合理的猜測，因為若沒有同步點，CPU在GPU進行處理時會是以不同步的方式工作。在此要學習的是，記憶體複製帶有隱式同步，不需要呼叫以上的`cuda.synchronize`。

### 練習：將CPU函數加速為自訂CUDA Kernel

以下是可用為CPU ufunc的CPU純量函數`square_device`。您的任務是將之重構，使其運作如同`@cuda.jit`修飾器修飾過的CUDA kernel。

您可能認為用`@vectorize`讓這個函數在裝置上運行會簡單得多，確實沒錯。但這個機會可以讓您在進入更複雜且實際的案例之前，先練習一下我們至今介紹過的所有語法。

在此練習中您需要：
* 將`square_device`定義重構成CUDA kernel，使此kernel能夠在單一元素上達成一個thread的工作量。
* 將下方的`d_a`和`d_out`陣列重構成CUDA裝置陣列。
* 將`blocks`和`threads`變數修改為適合於已知`n`的數值。
* 將對`square_device`的呼叫重構成包含執行配置的Kernel。

直到您順利完成上述實施之前，以下的測試都不會成功。若有問題，請參考[解決方案](../../../../edit/tasks/task2/task/solutions/square_device_solution.py)。

In [5]:
# Refactor to be a CUDA kernel doing one thread's work.
# Don't forget that when using `@cuda.jit`, you must provide an output array as no value will be returned.
"""
def square_device(a):
    return a**2
"""

import numpy as np
from numba import cuda

@cuda.jit
def square_device(a, out):
    idx = cuda.grid(1)
    out[idx] = a[idx]**2

In [6]:
# Leave the values in this cell fixed for this exercise
n = 4096

a = np.arange(n)
out = a**2 # `out` will only be used for testing below

In [7]:
"""
d_a = a                  # TODO make `d_a` a device array
d_out = np.zeros_like(a) # TODO: make d_out a device array

# TODO: Update the execution configuration for the amount of work needed
blocks = 0
threads = 0

# TODO: Launch as a kernel with an appropriate execution configuration
d_out = square_device(d_a)

"""
d_a = cuda.to_device(a)
d_out = cuda.device_array(shape=(n,), dtype=np.float32)

threads = 32
blocks = 128

square_device[blocks, threads](d_a, d_out)

In [8]:
from numpy import testing
testing.assert_almost_equal(d_out, out)

## 關於隱藏延遲的附加資訊及執行配置選擇

支援CUDA的NVIDIA GPU在一個晶片上包含多個附加有DRAM的[**串流多重處理器(Streaming Multiprocessors)**](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#hardware-implementation)，又稱為**SM**，SM包含包含所有執行Kernel程式碼所需要的資源，包括許多CUDA核心。啟動Kernel時，每一Block只配給單一SM，但可能有多個Block分配給同一個SM。SM將Block分成32個執行緒子部分，稱為**執行緒束(Warp)**，而獲得待執行平行指令的正是這些執行緒束。

若一個指令的完成(或以CUDA用語來說是**到期(Expire)**)時間超過一個時脈週期，要是SM*還有可接受新指令的額外執行緒束，就可繼續進行工作*。由於SM上的暫存器檔案很大，SM在將發出指令之間的上下文改變成一個執行緒束或另一個時，不會產生時間損失。簡言之，只要還有其他還沒完成的工作，SM就可以用其他有意義的工作來隱藏操作延遲。

**因此，只要以夠大的Grid和Block尺寸執行Kernel，就能輕易對SM提供足夠數量的執行緒束，進而讓SM有能力隱藏延遲，如此方可發揮GPU潛能並藉此撰寫性能加速應用程式。**

決定CUDA Thread Grid的最佳大小是個複雜的問題，取決於演算法和特定GPU的[運算能力](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#compute-capabilities)兩方面，但在此，可從一些粗略地試探法著手：

  * Block的大小應是32個Thread(執行緒束尺寸)的倍數，一般Block的大小是每個Block中有128到512個Thread。
  * Grid大小應確保盡可能充分利用整個GPU。一開始可以啟用Block數量為GPU上SM數量2倍到4倍的Grid。一般而言，從20到100個之間Block的範圍會是很好的開始。
  * CUDA Kernel啟動開銷確實會隨著Block數量而增加，所以當輸入尺寸很大時，最好不要啟動Thread數量等於輸入元素數量的Grid，否則會產生極大量的Block。我們現在要改用另一種方式來處理大型輸入。

## Grid跨度迴圈處理大型資料集

以下投影片概略說明**Grid跨度迴圈(Grid Stride Loops)**，這種技巧所創造出的靈活Kernel中，每一Thread都能夠處理一個以上的資料元素，是處理大型資料集所必需的技巧。執行此單元以載入投影片。

In [9]:
from IPython.display import IFrame
IFrame('https://view.officeapps.live.com/op/view.aspx?src=https://developer.download.nvidia.com/training/courses/C-AC-02-V1-TW/AC_CUDA_Python_2-TW.pptx', 640, 390)

## 第一個Grid跨度迴圈

現在我們重構以上的`add_kernel`以利用Grid跨度迴圈，這樣我們就能啟動此Kernel，靈活地處理大型資料集，同時產生全域**記憶體合併**的益處，允許平行thread存取相鄰區塊中的記憶體，在此情況下，GPU可發揮減少記憶體操作總數的作用：

In [10]:
from numba import cuda

@cuda.jit
def add_kernel(x, y, out):
    

    start = cuda.grid(1)
    
    # This calculation gives the total number of threads in the entire grid
    stride = cuda.gridsize(1)   # 1 = one dimensional thread grid, returns a single value.
                                # This Numba-provided convenience function is equivalent to
                                # `cuda.blockDim.x * cuda.gridDim.x`

    # This thread will start work at the data element index equal to that of its own
    # unique index in the grid, and then, will stride the number of threads in the grid each
    # iteration so long as it has not stepped out of the data's bounds. In this way, each
    # thread may work on more than one data element, and together, all threads will work on
    # every data element.
    for i in range(start, x.shape[0], stride):
        # Assuming x and y inputs are same length
        out[i] = x[i] + y[i]

In [11]:
import numpy as np

n = 100000 # This is far more elements than threads in our grid
x = np.arange(n).astype(np.int32)
y = np.ones_like(x)

d_x = cuda.to_device(x)
d_y = cuda.to_device(y)
d_out = cuda.device_array_like(d_x)

threads_per_block = 128
blocks_per_grid = 30

In [12]:
add_kernel(d_x, d_y, d_out)
print(d_out.copy_to_host()) # Remember, memory copy carries implicit synchronization

[     1      2      3 ...  99998  99999 100000]


### 練習：實作Grid跨度迴圈

利用Grid跨度迴圈將以下CPU純量`hypot_stride`函數重構為可如同CUDA Kernel般運行。若有困難請參考[解決方案](../../../../edit/tasks/task2/task/solutions/hypot_stride_solution.py)。

In [15]:
"""  
from math import hypot
def hypot_stride(a, b, c):
    c = hypot(a, b)
"""
  
import numpy as np
from numba import cuda
from math import hypot

@cuda.jit
def hypot_stride(a, b, c):
    idx = cuda.grid(1)
    stride = cuda.gridsize(1)
    
    for i in range(idx, a.shape[0], stride):
        c[i] = hypot(a[i], b[i])

In [24]:
# You do not need to modify the contents in this cell
n = 1000000
a = np.random.uniform(-12, 12, n).astype(np.float32)
b = np.random.uniform(-12, 12, n).astype(np.float32)
d_a = cuda.to_device(a)
d_b = cuda.to_device(b)
d_c = cuda.device_array_like(d_b)

blocks = 128
threads_per_block = 64

hypot_stride[blocks, threads_per_block](d_a, d_b, d_c)
#hypot_stride[1, 1](d_a, d_b, d_c)

In [25]:
from numpy import testing
# This assertion will fail until you successfully implement the hypot_stride kernel above
testing.assert_almost_equal(np.hypot(a,b), d_c.copy_to_host(), decimal=5)

## Kernel計時

我們現在要花一點時間為`hypot_stride` Kernel安排效能測試的計時。若您之前未能順利實作，請在效能測試前先複製並執行[解決方案](../../../../edit/tasks/task2/task/solutions/hypot_stride_solution.py)。

### CPU 基準線

首先以`np.hypot`取得效能基準線：

In [26]:
%timeit np.hypot(a, b)

5.79 ms ± 5.66 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### CPU上的Numba

接著我們來看看CPU最佳化後的版本：

In [27]:
from numba import jit

@jit
def numba_hypot(a, b):
    return np.hypot(a, b)

In [28]:
%timeit numba_hypot(a, b)

5.51 ms ± 7.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### 裝置上的單一Thread控制

現在要在只有一個Thread的Grid中啟動我們的Kernel。我們使用的`%time`只會運行式一次，以確保我們的測量不會受到CUDA Kernel佇列有限深度的影響。我們也要加上`cuda.synchronize`，以保證我們在Kernel完成前不會因為將控制權交回計時器所在的CPU而獲得不正確的時間：

In [29]:
%time hypot_stride[1, 1](d_a, d_b, d_c); cuda.synchronize()

CPU times: user 168 ms, sys: 148 ms, total: 316 ms
Wall time: 316 ms


希望不會太意外，結果甚至比基準線CPU執行還慢。

### 裝置上的平行

In [30]:
%time hypot_stride[128, 64](d_a, d_b, d_c); cuda.synchronize()

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 546 µs


結果快多了！

## Atomic Operations與避免Race Conditions

CUDA就像許多通用平行執行架構一樣，可能會在程式碼中造成Race Condition。 當Thread進行讀寫的記憶體位置可能受到另一個獨立Thread所修改，CUDA中就會出現競爭條件。一般而言，需要擔心的是：

 * 寫入後讀取危害：一個Thread正在讀取一個記憶體位置，同時另一個Thread可能正在這個位置上進行寫入。
 * 寫入後寫入危害：兩個Thread同時對一個記憶體位置進行寫入，當Kernel完成時只會看得到一筆寫入。
 
避免上述兩種危害的常用策略是透過安排CUDA Kernel演算法，使得每一個Thread只負責獨特的一小組輸出陣列元素，及/或在單次Kernel呼叫中決不使用相同陣列進行輸入和輸出。(迭代演算法可視需要利用雙重緩衝策略，並在每次迭代上切換輸入與輸出陣列。)

但在許多情況下，不同Thread需要結合結果。以這個簡單的案例來說：「每個Thread皆增加一個全域計數器。」在您的Kernel中實作此事需要每一Thread：

1. 讀取全域計數器目前數值。
2. 運算`counter + 1`。
3. 將這個數值寫回全域記憶體。

但無法保證在步驟1到步驟3之間並沒有另一個Thread來改變全域計數器。為解決這個問題，CUDA提供的**原子操作**會以一個不可分割的步驟讀取、修改並更新記憶體位置。Numba支援多種函數，[說明於此](http://numba.pydata.org/numba-doc/dev/cuda/intrinsics.html#supported-atomic-operations)。

現在來製作我們的Thread計數器Kernel:

In [31]:
@cuda.jit
def thread_counter_race_condition(global_counter):
    global_counter[0] += 1  # This is bad
    
@cuda.jit
def thread_counter_safe(global_counter):
    cuda.atomic.add(global_counter, 0, 1)  # Safely add 1 to offset 0 in global_counter array

In [32]:
# This gets the wrong answer
global_counter = cuda.to_device(np.array([0], dtype=np.int32))
thread_counter_race_condition[64, 64](global_counter)

print('Should be %d:' % (64*64), global_counter.copy_to_host())

Should be 4096: [1]


In [33]:
# This works correctly
global_counter = cuda.to_device(np.array([0], dtype=np.int32))
thread_counter_safe[64, 64](global_counter)

print('Should be %d:' % (64*64), global_counter.copy_to_host())

Should be 4096: [4096]


## 評估

以下練習要請您充分運用至今所學。此處與先前練習的差異在於，沒有解決方案程式碼供您使用，並且您必須執行幾個額外的步驟以「執行評估」並取得您作答的分數。**開始處理前請仔細閱讀指示，以便順利完成評估。**

### 如何進行評估

執行以下步驟以完成評估：

1. 遵循以下指示，按照您一般進行練習的方式處理下方單元。
2. 達到您滿意的程度後，請遵循以下指示，複製程式碼並將之貼到連結的原始碼檔案中。貼入程式碼後請務必記得儲存檔案。
3. 回奧您用於啟動此本筆記的瀏覽器分頁，然後點擊**「評估(Assess)」**鈕。數秒後會顯示分數以及實用訊息。

歡迎隨時點擊**評估**鈕，若您首次未能通過，您可額外修改程式碼並重複執行步驟1至3。祝您好運！

### 撰寫加速直方圖Kernel

此項評估是請您建立一個加速直方圖Kernel。需要輸入資料陣列、範圍和窗口數，還要計算每個窗口中落入多少輸入資料元素。以下是一個可行的直方圖的CPU實作，供您做為參考範例：

In [37]:
def cpu_histogram(x, xmin, xmax, histogram_out):
    '''Increment bin counts in histogram_out, given histogram range [xmin, xmax).'''
    # Note that we don't have to pass in nbins explicitly, because the size of histogram_out determines it
    nbins = histogram_out.shape[0]
    bin_width = (xmax - xmin) / nbins
    
    # This is a very slow way to do this with NumPy, but looks similar to what you will do on the GPU
    for element in x:
        bin_number = np.int32((element - xmin)/bin_width)
        if bin_number >= 0 and bin_number < histogram_out.shape[0]:
            # only increment if in range
            histogram_out[bin_number] += 1

In [38]:
x = np.random.normal(size=10000, loc=0, scale=1).astype(np.float32)
xmin = np.float32(-4.0)
xmax = np.float32(4.0)
histogram_out = np.zeros(shape=10, dtype=np.int32)

cpu_histogram(x, xmin, xmax, histogram_out)

histogram_out

array([  12,   77,  455, 1535, 2856, 2952, 1560,  457,   89,    7],
      dtype=int32)

使用Grid跨度迴圈和原子操作，在下方單元中實作您的解決方案。完成任何修改後，執行評估前，請將此單元的內容貼[**`assessment/histogram.py`**](../../../../edit/tasks/task2/task/assessment/histogram.py)並儲存。

In [39]:
"""
@cuda.jit
def cuda_histogram(x, xmin, xmax, histogram_out):
    '''Increment bin counts in histogram_out, given histogram range [xmin, xmax).'''
    
    pass  # Replace this with your implementation
"""


@cuda.jit
def cuda_histogram(x, xmin, xmax, histogram_out):
    '''Increment bin counts in histogram_out, given histogram range [xmin, xmax).'''
    
    nbins = histogram_out.shape[0]
    bin_width = (xmax - xmin) / nbins
    
    start = cuda.grid(1)
    stride=cuda.gridsize(1)
    
    for i in range(start, x.shape[0], stride):
        bin_number = np.int32((x[i] - xmin)/bin_width)
        if bin_number >= 0 and bin_number < histogram_out.shape[0]:
            # only increment if in range
            cuda.atomic.add(histogram_out, bin_number, 1)

In [44]:
"""
d_x = cuda.to_device(x)
d_histogram_out = cuda.device_array_like(histogram_out)

blocks = 128
threads_per_block = 64

cuda_histogram[blocks, threads_per_block](d_x, xmin, xmax, d_histogram_out)
"""

d_x = cuda.to_device(x)
d_histogram_out = cuda.to_device(np.zeros(shape=10, dtype=np.int32))

blocks = 128
threads_per_block = 64

cuda_histogram[blocks, threads_per_block](d_x, xmin, xmax, d_histogram_out)

In [45]:
# This assertion will fail until you correctly implement `cuda_histogram`
np.testing.assert_array_almost_equal(d_histogram_out.copy_to_host(), histogram_out, decimal=2)

## 總結

在此段落中，您學到如何：

* 以Python 撰寫自訂CUDA Kernel並以執行執行配置。
* 使用Grid跨度迴圈在大型資料集上平行運作並發揮記憶體合併。
* 利用原子操作避免平行工作時出現競爭條件。

## 下載內容

如需下載本篇筆記內容，請執行以下單元，然後點擊下方的下載連結。請注意：若您是在本機 Jupyter伺服器中執行此notebook，notebook中的部分檔案路徑連結可能會因配合我們本身平台的資料夾結構而失效。您仍可透過Jupyter檔案瀏覽器查看檔案。

In [ ]:
!tar -zcvf section2.tar.gz .

[下載本段檔案。](files/section2.tar.gz)

## 附錄：問題查找與除錯

### 關於Terminal的注意事項

除錯是程式設計的重要環節。然而，出於很多原因，目前難以直接在Jupyter notebook內執行CUDA Kernel的除錯，所以此Notebook要使用Shell執行Jupyter notebook單元，顯示終端機指令。這些Shell指令會出現在notebook單元中，指令行前綴有`!`。使用此notebook中的除錯方法時，您應該可直接在Terminal執行指令。

### 列印

常用的除錯策略是列印。Numba支援CUDA kernel的列印，但有一些限制。請注意，Jupyter抓不到自CUDA kernel列印的輸出，所以需要使用可從Terminal運行的腳本來除錯。

以下是其中有錯的CUDA kernel：

In [ ]:
! cat debug/ex1.py

將此程式碼運行於直方圖 50值時，可以看到直方圖中並沒有50筆條目：

In [ ]:
! python debug/ex1.py

*(您可能已經發現錯誤所在，但我們現在要假裝不知道答案。)*

我們假定可能是窗口計算錯誤造成許多直方圖條目出現在範圍以外。我們在`if`陳述式周圍增加一些列印，看看到底發生什麼是：

In [ ]:
! cat debug/ex1a.py

這個Kernel會列出每一數值和其所計算的窗口數字。 從列印陳述式可以看到`print`支援常數字串和純量數值：

``` python
print('in range', x[i], bin_number)
```

不支援字串代換(使用C printf 語法或較新的`format()`語法)。 執行此腳本，可以看到：

In [ ]:
! python debug/ex1a.py

將該輸出掃描下來，我們看到50個數值全都應該在範圍內。 很顯然是一些類型的競爭條件改變了直方圖。事實上，嫌疑行是：

``` python
histogram_out[bin_number] += 1
```

其應為(如您在先前練習中所見)

``` python
cuda.atomic.add(histogram_out, bin_number, 1)
```

### CUDA模擬器

回顧早期的CUDA，`nvcc`設有可在CPU上執行CUDA碼以利除錯的「模擬器」模式。 有了`cuda-gdb`之後，新發佈的CUDA就捨棄了這項功能。 由於並沒有針對CUDA+Python的除錯器，所以Numba包含了一個「CUDA模擬器」，可用Python直譯器在主機CPU上執行您的CUDA碼。因此您可使用Python模組和之前編譯所不允許的函數來對您程式碼的邏輯進行除錯。

常見的使用案例是在CUDA Kernel的一個Thread內開始Python除錯器：
``` python
import numpy as np

from numba import cuda

@cuda.jit
def histogram(x, xmin, xmax, histogram_out):
    nbins = histogram_out.shape[0]
    bin_width = (xmax - xmin) / nbins

    start = cuda.grid(1)
    stride = cuda.gridsize(1)

    ### DEBUG FIRST THREAD
    if start == 0:
        from pdb import set_trace; set_trace()
    ###

    for i in range(start, x.shape[0], stride):
        bin_number = np.int32((x[i] + xmin)/bin_width)

        if bin_number >= 0 and bin_number < histogram_out.shape[0]:
            cuda.atomic.add(histogram_out, bin_number, 1)

x = np.random.normal(size=50, loc=0, scale=1).astype(np.float32)
xmin = np.float32(-4.0)
xmax = np.float32(4.0)
histogram_out = np.zeros(shape=10, dtype=np.int32)

histogram[64, 64](x, xmin, xmax, histogram_out)

print('input count:', x.shape[0])
print('histogram:', histogram_out)
print('count:', histogram_out.sum())
```

此程式碼可進行如同以下的除錯會話：
```
(gtc2017) 0179-sseibert:gtc2017-numba sseibert$ NUMBA_ENABLE_CUDASIM=1 python debug/ex2.py
> /Users/sseibert/continuum/conferences/gtc2017-numba/debug/ex2.py(18)histogram()
-> for i in range(start, x.shape[0], stride):
(Pdb) n
> /Users/sseibert/continuum/conferences/gtc2017-numba/debug/ex2.py(19)histogram()
-> bin_number = np.int32((x[i] + xmin)/bin_width)
(Pdb) n
> /Users/sseibert/continuum/conferences/gtc2017-numba/debug/ex2.py(21)histogram()
-> if bin_number >= 0 and bin_number < histogram_out.shape[0]:
(Pdb) p bin_number, x[i]
(-6, -1.4435024)
(Pdb) p x[i], xmin, bin_width
(-1.4435024, -4.0, 0.80000000000000004)
(Pdb) p (x[i] - xmin) / bin_width
3.1956219673156738
(Pdb) q
```

### CUDA記憶體檢查

另一個常見錯誤是CUDA Kernel的記憶體存取無效，通常是因為超出陣列末端。NVIDIA的完整CUDA工具套件(不是`cudatoolkit` conda套裝)包含一個名為`cuda-memcheck`的效用程式，其可檢查CUDA程式碼中的多種記憶體存取錯誤。

現在針對以下程式碼進行除錯：

In [ ]:
! cat debug/ex3.py

In [ ]:
! cuda-memcheck python debug/ex3.py

`cuda-memcheck`的輸出顯示我們的直方圖函數確實有問題：
```
========= Invalid __global__ write of size 4
=========     at 0x00000548 in cudapy::__main__::histogram$241(Array<float, int=1, C, mutable, aligned>, float, float, Array<int, int=1, C, mutable, aligned>)
```
但我們不知道是哪一行。 為獲得更多關於錯誤的資訊，我們在編譯kernel時，可以將kernel改變成這樣，來開啟「除錯」模式：
``` python
@cuda.jit(debug=True)
def histogram(x, xmin, xmax, histogram_out):
    nbins = histogram_out.shape[0]
```

In [ ]:
! cuda-memcheck python debug/ex3a.py

現在我們看到的錯誤訊息包含來源檔案和行號：`ex3a.py:17`。

In [ ]:
! cat -n debug/ex3a.py | grep -C 2 "17"

現在我們可能發現 if 陳述式中的`or`應為`and`。

`cuda-memcheck`具有不同模式，用以偵測不同種類的問題(類似於用來去除CPU記憶體存取錯誤的`valgrind`)。 至文件查看詳細資訊：http://docs.nvidia.com/cuda/cuda-memcheck/

## 附錄：CUDA參考資料

建議於完成本課程後，再來研讀CUDA C程式設計指南的第一章和第二章。雖是為CUDA C所寫，但也很適合用來程式設計CUDA Python。

 * 介紹：http://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#introduction
 * 程式設計模型：http://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#programming-model

## 附錄：以Numba在GPU上進行隨機產生數字

GPU對於需要使用大量隨機數字的蒙地卡羅應用程式而言非常實用。 CUDA的cuRAND庫中包含一組好用的隨機數字生成演算法。但不巧的是，cuRAND 是定義在Numba無法輕易編譯或連結的C標頭檔中。(Numba的CUDA JIT根本不會為CUDA kernel產生C程式碼。)Numba有計畫要解決這個問題，但可能需要花些時間。

同時，Numba版本0.33和以上版本包含`xoroshiro128+`生成器，雖然期間($2^{128} - 1$)短於cuRAND中的XORWOW生成器，但其品質極佳。

使用前，要先在主機上就Kernel中的每個Thread進行RNG狀態的初始化。這個狀態創造函數初始化在相同序列中的每個狀態，序列以Seed值命名，但彼此之間隔開$2^{64}$步。如此可確保不同Thread不會意外地以重疊序列結束(除非單一Thread產生$2^{64}$個隨機數字，你應該沒有耐心等待)：

In [ ]:
import numpy as np
from numba import cuda
from numba.cuda.random import create_xoroshiro128p_states, xoroshiro128p_uniform_float32

threads_per_block = 64
blocks = 24
rng_states = create_xoroshiro128p_states(threads_per_block * blocks, seed=1)

我們可將Kernel中的這些隨機數字狀態當成引數傳出：

In [ ]:
@cuda.jit
def monte_carlo_mean(rng_states, iterations, out):
    thread_id = cuda.grid(1)
    total = 0
    for i in range(iterations):
        sample = xoroshiro128p_uniform_float32(rng_states, thread_id) # Returns a float32 in range [0.0, 1.0)
        total += sample
        
    out[thread_id] = total/iterations

In [ ]:
out = cuda.device_array(threads_per_block * blocks, dtype=np.float32)
monte_carlo_mean[blocks, threads_per_block](rng_states, 10000, out)
print(out.copy_to_host().mean())

### 練習：GPU上的蒙地卡羅Pi

現在回頭看第一段中，我們用Numba在CPU上編譯的蒙地卡羅Pi演算法。

In [ ]:
from numba import njit
import random

@njit
def monte_carlo_pi(nsamples):
    acc = 0
    for i in range(nsamples):
        x = random.random()
        y = random.random()
        if (x**2 + y**2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples

In [ ]:
nsamples = 10000000
%timeit monte_carlo_pi(nsamples)

您的任務是將目前與上方`monte_carlo_pi_device`相同的以下`monte_carlo_pi`重構為在GPU上運行。您可參考上方的`monte_carlo_mean`，但您至少需要：

- 修飾為CUDA Kernel
- 從裝置RNG狀態取出Thread樣本(下方兩個單元產生)
- 將每個Thread的結果儲存於應在主機上的輸出陣列中(如上文`monte_carlo_mean`所做)

從下方兩個單元可看出所有資料都已經初始化，執行組態建立，且啟動Kernel。您要做的就是重構下方單元中的Kernel定義。若有問題請參考[解決方案](../../../../edit/tasks/task3/task/solutions/monte_carlo_pi_solution.py)。

In [ ]:
from numba import njit
import random

# TODO: All your work will be in this cell. Refactor to run on the device successfully given the way the
# kernel is launched below.
@njit
def monte_carlo_pi_device(nsamples):
    acc = 0
    for i in range(nsamples):
        x = random.random()
        y = random.random()
        if (x**2 + y**2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples

In [ ]:
# Do not change any of the values in this cell
nsamples = 10000000
threads_per_block = 128
blocks = 32

grid_size = threads_per_block * blocks
samples_per_thread = int(nsamples / grid_size) # Each thread only needs to work on a fraction of total number of samples.
                                               # This could also be calcuated inside the kernel definition using `gridsize(1)`.

rng_states = create_xoroshiro128p_states(grid_size, seed=1)
d_out = cuda.device_array(threads_per_block * blocks, dtype=np.float32)

In [ ]:
%time monte_carlo_pi_device[blocks, threads_per_block](rng_states, samples_per_thread, d_out); cuda.synchronize()

In [ ]:
print(d_out.copy_to_host().mean())

<a href="https://www.nvidia.com/dli"> <img src="images/DLI Header.png" alt="Header" style="width: 400px;"/> </a>